In [20]:
import pandas as pd
import numpy as np
from sklearn_pandas import DataFrameMapper
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.svm import SVC
# from sklearn.model_selection import train_test_split
# from sklearn.linear_model import LogisticRegression
# from sklearn.neighbors import KNeighborsClassifier
# from sklearn.naive_bayes import GaussianNB

In [2]:
dict_sex={'Male':0,'Female':1}

# Single = 0 / In current relationship = 1
dict_status={'Never-married':0, 'Married-civ-spouse':1, 'Divorced':0,
       'Married-spouse-absent':1, 'Separated':0, 'Married-AF-spouse':1,
       'Widowed':0}

dict_region={'United-States':'NorthAmerica', 'Cuba':'SouthAmerica', 'Jamaica':'SouthAmerica', 'India':'Asia',
'Mexico':'SouthAmerica','South':'Asia', 'Puerto-Rico':'SouthAmerica', 
'Honduras':'SouthAmerica', 'England':'Europe', 'Canada':'NorthAmerica', 
'Germany':'Europe','Iran':'Asia', 'Philippines':'Asia', 'Italy':'Europe',
'Poland':'Europe', 'Columbia':'SouthAmerica','Cambodia':'Asia',
'Thailand':'Asia', 'Ecuador':'SouthAmerica', 'Laos':'Asia', 'Taiwan':'Asia', 
'Haiti':'SouthAmerica','Portugal':'Europe','Dominican-Republic':'SouthAmerica',
'El-Salvador':'SouthAmerica', 'France':'Europe', 'Guatemala':'SouthAmerica',
'China':'Asia', 'Japan':'Asia', 'Yugoslavia':'Europe', 'Peru':'SouthAmerica',
'Outlying-US(Guam-USVI-etc)':'NorthAmerica', 'Scotland':'Europe', 'Trinadad&Tobago':'SouthAmerica',
'Greece':'Europe', 'Nicaragua':'SouthAmerica', 'Vietnam':'Asia', 'Hong':'Asia', 'Ireland':'Europe', 
'Hungary':'Europe','Holand-Netherlands':'Europe'}

# Working for government =1 / working in private or not-working =0
dict_workclass={'State-gov':1, 'Self-emp-not-inc':0, 'Private':0, 'Federal-gov':1,
'Local-gov':1, 'Self-emp-inc':0, 'Without-pay':0,'Never-worked':0}


In [17]:
## Train/test sets
df = pd.read_csv('Training_census_data.csv',index_col=False)
df_train=df.copy()

df_train.replace('?',np.nan,inplace=True)
df_train.sex.replace(dict_sex,inplace=True)
df_train.marital_status.replace(dict_status,inplace=True)
df_train.native_country.replace(dict_region,inplace=True)
df_train.workclass.replace(dict_workclass,inplace=True)
df_train.income.replace({'<=50K':0,'>50K':1},inplace=True)


df_train.loc[(df_train.Age>=df_train.Age.min()) & (df_train.Age< 25), 'Age_range'] = 'Young' 
df_train.loc[(df_train.Age>=25) & (df_train.Age< 40), 'Age_range'] = 'Adult1'
df_train.loc[(df_train.Age>=40) & (df_train.Age< 65), 'Age_range'] = 'Adult2'
df_train.loc[(df_train.Age>=65) & (df_train.Age< df_train.Age.max()), 'Age_range'] = 'Senior'
 
df_train.loc[(df_train.hours_per_week<df_train.hours_per_week.median()), 'hours_range'] = 0 
df_train.loc[(df_train.hours_per_week>=df_train.hours_per_week.median()), 'hours_range'] = 1

df_train=df_train.dropna().reset_index(drop=True)

mapper_df = DataFrameMapper([('occupation', CountVectorizer(token_pattern=r"[\w\b-]+")),
                            ('race', CountVectorizer(token_pattern=r"[\w\b-]+")),
                            ('native_country', CountVectorizer()),
                            ('Age_range', CountVectorizer())], df_out=True)
df_vectorized=mapper_df.fit_transform(df_train.copy())


X = df_train[['workclass','education_num','marital_status','sex',
                     'capital_gain','capital_loss','hours_range']].join(df_vectorized)

y=df_train['income']

# X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=0)

clf = SVC(C=0.2, gamma=0.1,random_state=0).fit(X, y)

In [33]:
# clf = LogisticRegression(C=0.2, solver='liblinear',random_state=0).fit(X_train, y_train)
# clf.score(X_test, y_test) #0.845

# clf = KNeighborsClassifier(n_neighbors=5).fit(X_train, y_train)
# clf.score(X_test, y_test) #0.84

# clf = GaussianNB().fit(X_train, y_train)
# clf.score(X_test, y_test) #0.82

In [19]:
# To predict set
df_pred = pd.read_csv('Predicted_census_data.csv',index_col=False)

df_pred.replace('?',np.nan,inplace=True)
df_pred.sex.replace(dict_sex,inplace=True)
df_pred.marital_status.replace(dict_status,inplace=True)
df_pred.native_country.replace(dict_region,inplace=True)
df_pred.workclass.replace(dict_workclass,inplace=True)
df_pred.income.replace({'<=50K.':0,'>50K.':1},inplace=True)


df_pred.loc[(df_pred.Age>=df_pred.Age.min()) & (df_pred.Age< 25), 'Age_range'] = 'Young' 
df_pred.loc[(df_pred.Age>=25) & (df_pred.Age< 40), 'Age_range'] = 'Adult1'
df_pred.loc[(df_pred.Age>=40) & (df_pred.Age< 65), 'Age_range'] = 'Adult2'
df_pred.loc[(df_pred.Age>=65) & (df_pred.Age< df_pred.Age.max()), 'Age_range'] = 'Senior'
 
df_pred.loc[(df_pred.hours_per_week<df_pred.hours_per_week.median()), 'hours_range'] = 0 
df_pred.loc[(df_pred.hours_per_week>=df_pred.hours_per_week.median()), 'hours_range'] = 1

df_pred=df_pred.dropna().reset_index(drop=True)

mapper_dfp = DataFrameMapper([('occupation', CountVectorizer(token_pattern=r"[\w\b-]+")),
                            ('race', CountVectorizer(token_pattern=r"[\w\b-]+")),
                            ('native_country', CountVectorizer()),
                            ('Age_range', CountVectorizer())], df_out=True)
df_vectorizedp=mapper_df.fit_transform(df_pred.copy())


X_pred = df_pred[['workclass','education_num','marital_status','sex',
                     'capital_gain','capital_loss','hours_range']].join(df_vectorizedp)

y_pred=df_pred['income']

clf.score(X_pred, y_pred) 


0.8611867898199216